In [11]:
import pytorch_lightning as pl
import numpy as np
import torchaudio
import torch
import torch.optim
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoderLayer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from utils import WaveNet, AudioMELSpectogramDataset, CalcContentLoss, CalcStyleLoss 

In [12]:
torch.set_float32_matmul_precision('medium')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [13]:
class StyleAutoEncoder(nn.Module):

    def __init__(self, in_channels, shared_channels, out_channels, kernel_size, dilation, nhead=8, dropout=0.2, dim_feedforward=1024):
        super(StyleAutoEncoder, self).__init__()
        self.sharedEncoder = WaveNet(in_channels, shared_channels, kernel_size, dilation, dropout)
        self.speechEncoder = WaveNet(shared_channels, out_channels, kernel_size, dilation, dropout)
        self.styleEncoder = WaveNet(shared_channels, out_channels, kernel_size, dilation, dropout)
        self.style_encoder_attention = TransformerEncoderLayer(
            d_model=out_channels,
            nhead=8,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.speech_encoder_attention = TransformerEncoderLayer(
            d_model=out_channels,
            nhead=8,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.decoder_attention = TransformerDecoderLayer(
            d_model=out_channels*2,
            nhead=8,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.decoder = WaveNet(out_channels*2, 1, kernel_size, dilation, dropout)

    def forward(self, speech, style):

        # print('speech', speech.shape)
        # print('style', style.shape)

        style = self.sharedEncoder(style)
        speech = self.sharedEncoder(speech) 

        style = self.styleEncoder(style)
        speech = self.speechEncoder(speech)
        
        style = style.permute(2, 0, 1)  # Reordenamos las dimensiones para que sea compatible con la capa de atención
        style_encoder = self.style_encoder_attention(style)
        style = style.permute(1, 2, 0)
        style_encoder = speech.permute(0, 1, 2)

        speech = speech.permute(2, 0, 1)
        speech_encoder = self.speech_encoder_attention(speech)
        speech = speech.permute(1, 2, 0)
        speech_encoder = speech.permute(0, 1, 2)
        
        # print('speech', speech.shape)
        # print('style', style.shape)
        out = torch.cat((speech, style), dim=1)
        # print('speech_encoder', speech_encoder.shape)
        # print('style_encoder', style_encoder.shape)
        out_encoder = torch.cat((speech_encoder, style_encoder), dim=1)
        
        out = out.permute(2, 0, 1) 
        out_encoder = out.permute(0, 1, 2) 

        # print('out', out.shape)
        # print('out_encoder', out_encoder.shape)

        x = self.decoder_attention(out, out_encoder)
        x = x.permute(1, 2, 0) 

        # print('x', x.shape)

        x = self.decoder(x)
        # x = x.unsqueeze(1)
        # print('x', x.shape)

        return x #.squeeze(0)
 
class MelSpecVCAutoencoderModule(pl.LightningModule):

    def __init__(self, in_channels, shared_channels, out_channels, kernel_size, dilation, device, sample_rate, dropout=0.2):
        super(MelSpecVCAutoencoderModule, self).__init__()
        self.encoder_generator = StyleAutoEncoder(in_channels, shared_channels, out_channels, kernel_size, dilation, dropout)
        self.todevice = device
        self.speech_weight = 1
        self.style_weight = 100

    def forward(self, speech, style):
        y = self.encoder_generator(speech, style)
        return y

    def debug(self, loss_speech, loss_style, loss):
        # print('loss_speech', loss_speech.item())
        # print('loss_style', loss_style.item())
        # print('shape', loss.shape)
        # print('loss', loss.item())
        pass

    def apply_loss(self, batch):
        speech, style = batch
        if speech.ndim == 2: speech = speech.unsqueeze(1)
        if style.ndim == 2: style = style.unsqueeze(1)
        y = self(speech, style)
        loss_speech = CalcContentLoss(y, speech) * self.speech_weight
        loss_style = CalcStyleLoss(y, style) * self.style_weight
        loss = loss_speech + loss_style
        self.debug(loss_speech, loss_style, loss)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.apply_loss(batch)
        self.log('train_loss', loss.item())
        return loss 
    
    def validation_step(self, batch, batch_idx):
        loss = self.apply_loss(batch)
        self.log('val_loss', loss.item())
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

In [14]:
batch_size = 32
sample_rate = 16000
div_ratio = 0.8
n_fft = 800
audio_length = 3
in_channels, shared_channels, out_channels, kernel_size, dilation = 128, 64, 32, 3, 12
metadata = pd.read_csv('metadata.csv')
dataset = AudioMELSpectogramDataset(metadata, device, sample_rate=sample_rate, audio_length=audio_length, n_fft=n_fft)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(len(dataset) * div_ratio), len(dataset) - int(len(dataset) * div_ratio)])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=True)
model = MelSpecVCAutoencoderModule(in_channels, shared_channels, out_channels, kernel_size, dilation, device=device, sample_rate=sample_rate)
model = model.to(device)
model = model.to(torch.float64)

e:\miniconda3\envs\AI\lib\site-packages\torchaudio\functional\functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [15]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='./checkpoints',
    filename='{epoch}-{val_loss:.2f}-{val_r2:.2f}',
    save_top_k=1,
    monitor='val_loss',
    every_n_epochs =1,
    mode='min'
)

trainer = pl.Trainer(
    accelerator='gpu',
    max_epochs=10,
    benchmark=True,
    # deterministic=True,
    precision=16,
    accumulate_grad_batches=6,
    callbacks=[
        checkpoint_callback,
        pl.callbacks.LearningRateMonitor(logging_interval='step')
    ]
)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
# trainer.fit(model, train_loader, val_loader)

In [17]:
def Mel_to_audio_torch(mel_spectrogram, sample_rate):
    linear_spectrogram = torchaudio.transforms.InverseMelScale(n_stft=1024)(mel_spectrogram.float())

    # Convert linear spectrogram to audio
    audio = torchaudio.transforms.GriffinLim(n_fft=1024)(linear_spectrogram)

    return audio.squeeze().numpy()

random_speech = train_dataset[120]
mel_spectrogram = random_speech[0].to(device)
style_spec = random_speech[1].to(device)

print(mel_spectrogram.shape, style_spec.shape)

audio_wav = model(mel_spectrogram, style_spec)

display(ipd.Audio(audio_wav.cpu().numpy(), rate=sample_rate))

torch.Size([128, 241]) torch.Size([128, 241])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:15                                                                                   │
│                                                                                                  │
│   12                                                                                             │
│   13 print(mel_spectrogram.shape, style_spec.shape)                                              │
│   14                                                                                             │
│ ❱ 15 audio_wav = model(mel_spectrogram, style_spec)                                              │
│   16                                                                                             │
│   17 display(ipd.Audio(audio_wav.cpu().numpy(), rate=sample_rate))                               │
│   18                                                                                             │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\torch\nn\modules\module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ in forward:86                                                                                    │
│                                                                                                  │
│    83 │   │   self.style_weight = 100                                                            │
│    84 │                                                                                          │
│    85 │   def forward(self, speech, style):                                                      │
│ ❱  86 │   │   y = self.encoder_generator(speech, style)                                          │
│    87 │   │   return y                                                                           │
│    88 │                                                                                          │
│    89 │   def debug(self, loss_speech, loss_style, loss):                                        │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\torch\nn\modules\module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                     